# Proyecto ISIS-3302 (Modelado, Optimización y Simulación)

- Paulina Arrazola Vernaza - 202020631
- Santiago Alejandro Jaimes Puerto - 201912921
- Nicolás Rincón Sánchez - 202021963

# Etapa II (Implementación)

## 1. Configuración y Procesamiento

### 1.1 Importación de Librerías

En el desarrollo de la implementación del proyecto, se requiere la instalación de librerías especializadas. En particular, se emplea como framework de optimización a **Pyomo** y se utilizan librerías para análisis de datos, como **Pandas**.

In [5]:
import pyomo.environ as pyo
import pandas as pd
import requests
import json
import time
import sys
import math
import os
import itertools
from collections import OrderedDict

### 1.2 Configuración y Carga de datos

Para la extracción de datos de distancia por la calle (trayectos vehiculares sobre la malla vial de la ciudad) entre dos puntos A y B, se utilizó el API de OSRM. A continuación, se deja preestablecida la carga de los datos del caso 1.

In [6]:
# URL del servidor OSRM al cual se le harán las peticiones
OSRM_URL   = "https://router.project-osrm.org/table/v1/driving/"

# Costo por kilómetro definido según nuestra investigación en la Etapa I
C_KM = 20700 # COP/km

# Carga de los datos
depots = {}
clients = {}
vehicles = {}

for i in range(1,4):
    try:
        depots[i] = pd.read_csv(f"case_{i}_base/Depots.csv")
        clients[i] = pd.read_csv(f"case_{i}_base/Clients.csv")
        vehicles[i] = pd.read_csv(f"case_{i}_base/Vehicles.csv")
        print(f"Archivos CSV cargados correctamente para el Caso {i}.")

        # Se agregan variables a cada base de datos en el workspace global
        globals()[f"depots_case_{i}_df"] = depots[i]
        globals()[f"clients_case_{i}_df"] = clients[i]
        globals()[f"vehicles_case_{i}_df"] = vehicles[i]

    except FileNotFoundError as e:
        print(f"No se encontró archivo CSV: {e}", file=sys.stderr)
        sys.exit(1)
    except Exception as e:
        print(f"Un error inesperado sucedió durante la carga: {e}", file=sys.stderr)
        sys.exit(1)


Archivos CSV cargados correctamente para el Caso 1.
Archivos CSV cargados correctamente para el Caso 2.
Archivos CSV cargados correctamente para el Caso 3.


### 1.3 Preprocesamiento de Datos

Se realiza el procesamiento de los datos obtenidos desde el CSV para dejarlos en un formato propio de Python utilizable en futuros pasos.

In [7]:
# Definir una estructura que contenga todos los datos preparados por cada caso
prepared_data = {}

for case in range(1,4):
    # Extraer los DataFrames de cada caso
    depots_df = globals()[f'depots_case_{case}_df']
    clients_df = globals()[f'clients_case_{case}_df']
    vehicles_df = globals()[f'vehicles_case_{case}_df']

    # Conjuntos para Pyomo
    set_I_data = [f'D{depot_id}' for depot_id in depots_df['DepotID']]
    set_J_data = [f'C{client_id}' for client_id in clients_df['ClientID']]
    set_K_data = [f'V{i+1}' for i in range(len(vehicles_df))]
    set_N_data = set_I_data + set_J_data

    # Parámetros para Pyomo
    if 'Capacity' not in depots_df.columns:
        # Cuando no haya capacidad, le ponemos capacidad infinita
        depots_df['Capacity'] = float('inf')
        print("Columna 'Capacity' no encontrada — todas las capacidades de los depósitos se establecen en infinito.")

    param_A_data = {f'D{depots_df.loc[i, "DepotID"]}': depots_df.loc[i, 'Capacity'] 
                    for i in depots_df.index}
    param_D_data = {f'C{clients_df.loc[i, "ClientID"]}': clients_df.loc[i, 'Product']
                    for i in clients_df.index}
    param_Q_data = {f'V{i+1}': vehicles_df.loc[i, 'Capacity'] for i in vehicles_df.index}
    param_R_data = {f'V{i+1}': vehicles_df.loc[i, 'Range'] for i in vehicles_df.index}
    param_n_cust_data = len(set_J_data)

    # Diccionario de datos para la construcción de la matriz de distancias
    data = {}
    data['N'] = set_N_data # Conjunto de nodos (I + J)
    data['UbicacionNodo'] = {} # Coordenadas {NodeID: (lat, lon)}
    data['ID'] = list(vehicles_df['VehicleID'].unique()) # List de tipos únicos de vehículo (por los datos que nos entregan)

    # Poblar el diccionario de coordenadas (lat, lon)
    for i in depots_df.index:
        node_id = f'D{depots_df.loc[i, "DepotID"]}'
        data['UbicacionNodo'][node_id] = (depots_df.loc[i, 'Latitude'], depots_df.loc[i, 'Longitude'])
    for i in clients_df.index:
        node_id = f'C{clients_df.loc[i, "ClientID"]}'
        data['UbicacionNodo'][node_id] = (clients_df.loc[i, 'Latitude'], clients_df.loc[i, 'Longitude'])

    # Inicialización del diccionario 
    data['d'] = {tv: {u: {} for u in data['N']} for tv in data['ID']}

    # Placeholder para Pyomo (se va a llenar con lo que arroja OSRM)
    param_d_data = {} # Contendrá {(u, v): distancia_km}
    param_c_data = {} # Contendrá {(u, v): costo_ruta_cop}

    print("--- Datos preparados para Pyomo y cálculo de matriz de distancias ---")
    print("======== CONJUNTOS ======")
    print(f"Set I (Depositos): {set_I_data}")
    print(f"Set J (Clientes): {set_J_data}")
    print(f"Set K (Vehículos): {set_K_data}")
    print(f"Set N (Nodos): {data['N']}")
    print("======= PARÁMETROS ========")
    print(f"Param A (Capacidades de depósitos): {param_A_data}")
    print(f"Param D (Demandas): {param_D_data}")
    print(f"Param Q (Veh. capacidad): {param_Q_data}")
    print(f"Param R (Veh. rango): {param_R_data}") 
    print(f"Param n_cust (No. clientes): {param_n_cust_data}")
    print("======== UBICACIONES DE NODOS =========")
    print(f"UbicacionNodo: {data['UbicacionNodo']}")

    # Almacenar los datos en estructuras globales
    globals()[f'set_I_data{case}'] = set_I_data
    globals()[f'set_J_data{case}'] = set_J_data
    globals()[f'set_K_data{case}'] = set_K_data
    globals()[f'set_N_data{case}'] = set_N_data

    globals()[f'param_A_data{case}'] = param_A_data
    globals()[f'param_D_data{case}'] = param_D_data
    globals()[f'param_Q_data{case}'] = param_Q_data
    globals()[f'param_R_data{case}'] = param_R_data
    globals()[f'param_n_cust_data{case}'] = param_n_cust_data

    globals()[f'data{case}'] = data
    globals()[f'param_d_data{case}'] = param_d_data
    globals()[f'param_c_data{case}'] = param_c_data

    print(f" Datos procesados para el Caso {case} \n\n")

Columna 'Capacity' no encontrada — todas las capacidades de los depósitos se establecen en infinito.
--- Datos preparados para Pyomo y cálculo de matriz de distancias ---
======== CONJUNTOS ======
Set I (Depositos): ['D1']
Set J (Clientes): ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
Set K (Vehículos): ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']
Set N (Nodos): ['D1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
======= PARÁMETROS ========
Param A (Capacidades de depósitos): {'D1': inf}
Param D (Demandas): {'C1': 13, 'C2': 15, 'C3': 12, 'C4': 15, 'C5': 20, 'C6': 17, 'C7': 17, 'C8': 20, 'C9': 20, 'C10': 15, 'C11': 17, 'C12': 12, 'C13': 21, 'C14': 1

### 1.4 Generación de la Matriz de distancias

En este paso, se calcula las distancias entre todos los nodos del grafo que incluye a los depósitos, los centros de distribución y los destinos. Recordemos que, como se mencionó al inicio, se utiliza el API de OSRM. Sin embargo, debido a que el API tiene una limitación respecto a la cantidad de nodos que puede procesar en una sola consulta, se tomó la decisión de dividir el conjunto de datos en pequeños batches.

Adicionalmente, se emplea un archivo de caché para evitar llamados redundantes al API.

In [8]:
# Límite de coordenadas por consulta OSRM
MAX_COORDS  = 100  

for case in range(1, 4):
    print(f"\n======= CÁLCULO DE DISTANCIAS CASO {case} ===========")
    
    # Inicializar variables y cargar datos del caso
    param_d_data = globals()[f'param_d_data{case}']
    param_c_data = globals()[f'param_c_data{case}']
    
    # Variables de configuración
    CACHE_FILE = f'osrm_cache_case{case}.json'


    # Construir lista de nodos y diccionario de coordenadas
    N = globals()[f'data{case}']['N']
    coords = globals()[f'data{case}']['UbicacionNodo']


    # Cargar caché existente (claves "u|v")
    dist_cache = {}
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE) as f:
                raw = json.load(f)
            dist_cache = {tuple(k.split("|")): v for k, v in raw.items()}
            print(f"  Distancias cargadas de caché ({len(dist_cache)} pares).")
        except json.JSONDecodeError:
            print("  Caché dañada, se ignorará.")
            dist_cache = {}


    # Identificar pares faltantes y consultar OSRM
    all_pairs = set(itertools.product(N, N))
    missing_pairs = [p for p in all_pairs if p not in dist_cache]

    if missing_pairs:
        print(f"Faltan {len(missing_pairs)} pares. Calculando distancias…")

        if len(N) > MAX_COORDS:
            
            # Busca por batches origen×destino (si N > MAX_COORDS)
            BATCH_SIZE    = max(1, MAX_COORDS // 2)
            total_batches = math.ceil(len(N) / BATCH_SIZE) ** 2
            batch_no      = 0

            for i_start in range(0, len(N), BATCH_SIZE):
                sources = N[i_start : i_start + BATCH_SIZE]
                for j_start in range(0, len(N), BATCH_SIZE):
                    batch_no += 1
                    dests = N[j_start : j_start + BATCH_SIZE]
                    print(f"  Batch {batch_no}/{total_batches}: {len(sources)}×{len(dests)}")

                    # Solo pares que faltan en este bloque
                    current_missing = [
                        (u, v) for (u, v) in missing_pairs
                        if u in sources and v in dests
                    ]
                    if not current_missing:
                        continue

                    # Construir consulta OSRM
                    query_nodes = sorted(set(sources + dests))
                    idx_map     = {n: idx for idx, n in enumerate(query_nodes)}
                    coord_str   = ";".join(f"{coords[n][1]},{coords[n][0]}" for n in query_nodes)
                    params      = {
                        "sources":      ";".join(str(idx_map[u]) for u in sources),
                        "destinations": ";".join(str(idx_map[v]) for v in dests),
                        "annotations":  "distance",
                    }

                    try:
                        r = requests.get(OSRM_URL + coord_str, params=params, timeout=60)
                        r.raise_for_status()
                        matrix = r.json().get("distances", [])
                        if not matrix:
                            raise ValueError("Matriz vacía")
                    except Exception as e:
                        print(f"Error OSRM (batch {batch_no}): {e}, asignando infinito")
                        for u, v in current_missing:
                            dist_cache[(u, v)] = math.inf
                        continue

                    # Rellenar dist_cache con km
                    for u, v in current_missing:
                        d = matrix[sources.index(u)][dests.index(v)]
                        dist_cache[(u, v)] = math.inf if d is None else d / 1000.0

            # Guardar caché actualizado y ordenarlo
            depots    = sorted(n for n in N if n.startswith("D"))
            customers = sorted(n for n in N if n.startswith("C"))
            nodes     = depots + customers

            ordered_cache = OrderedDict()
            for u in nodes:
                for v in nodes:
                    ordered_cache[(u,v)] = dist_cache.get((u, v), math.inf)
            
            dist_cache = ordered_cache

        else:
            
            #Búsqueda de todo (si N ≤ MAX_COORDS)
            print(f"  N={len(N)} ≤ {MAX_COORDS}: trayendo todos desde OSRM")
            for start in range(0, len(N), MAX_COORDS):
                sub_nodes = N[start : start + MAX_COORDS]
                coord_str  = ";".join(f"{coords[n][1]},{coords[n][0]}" for n in sub_nodes)

                try:
                    r = requests.get(OSRM_URL + coord_str,
                                    params={"annotations": "distance"},
                                    timeout=60)
                    r.raise_for_status()
                    matrix = r.json().get("distances", [])
                    if not matrix:
                        raise ValueError("Matriz vacía")
                except Exception as e:
                    print(f"Error OSRM (bloque {start}): {e}, asignando infinito")
                    matrix = [[math.inf]*len(sub_nodes) for _ in sub_nodes]

                for i, u in enumerate(sub_nodes):
                    for j, v in enumerate(sub_nodes):
                        d = matrix[i][j] if matrix else None
                        dist_cache[(u, v)] = math.inf if d is None else d / 1000.0

        # Almancenar el caché en los archivos
        with open(CACHE_FILE, "w") as f:
            json.dump({f"{u}|{v}": km for (u, v), km in dist_cache.items()}, f)
        print("Caché OSRM actualizada.")
    else:
        print("No faltaban distancias nuevas.")


    # Rellenar param_d_data y param_c_data para Pyomo
    param_d_data.clear()
    param_c_data.clear()
    for u in N:
        for v in N:
            km = dist_cache.get((u, v), math.inf)
            param_d_data[u, v] = km
            param_c_data[u, v] = km * C_KM

    print(f"Matriz de distancias y costos lista ({len(N)}x{len(N)} pares).")

    # Mostrar un subconjunto de ejemplo para confirmar el correcto funcionamiento del código
    print("\nMuestra (primeros 4 nodos):")
    for u in N[:4]:
        for v in N[:4]:
            print(f"d({u},{v})={param_d_data[u,v]:.2f} km, "
                f"c={param_c_data[u,v]:,.0f}", end="  |  ")
        print()



======= CÁLCULO DE DISTANCIAS CASO 1 ===========
  Distancias cargadas de caché (625 pares).
No faltaban distancias nuevas.
Matriz de distancias y costos lista (25x25 pares).

Muestra (primeros 4 nodos):
d(D1,D1)=0.00 km, c=0  |  d(D1,C1)=23.34 km, c=483,177  |  d(D1,C2)=8.91 km, c=184,369  |  d(D1,C3)=7.97 km, c=165,008  |  
d(C1,D1)=22.72 km, c=470,306  |  d(C1,C1)=0.00 km, c=0  |  d(C1,C2)=14.26 km, c=295,105  |  d(C1,C3)=19.40 km, c=401,586  |  
d(C2,D1)=10.13 km, c=209,741  |  d(C2,C1)=12.67 km, c=262,315  |  d(C2,C2)=0.00 km, c=0  |  d(C2,C3)=6.81 km, c=141,021  |  
d(C3,D1)=11.32 km, c=234,353  |  d(C3,C1)=15.95 km, c=330,169  |  d(C3,C2)=6.48 km, c=134,198  |  d(C3,C3)=0.00 km, c=0  |  

======= CÁLCULO DE DISTANCIAS CASO 2 ===========
  Distancias cargadas de caché (441 pares).
No faltaban distancias nuevas.
Matriz de distancias y costos lista (21x21 pares).

Muestra (primeros 4 nodos):
d(D1,D1)=0.00 km, c=0  |  d(D1,D2)=33.06 km, c=684,367  |  d(D1,D3)=10.16 km, c=210,326  |

## 2. Modelado y Optimización

### 2.1 Modelo Matematico en Pyomo

Se genera el modelo planteado por nosotros siguiendo los pasos de inicialización en Pyomo:
- Conjuntos
- Parámetros
- Variables de Decisión
- Restricciones

Debido a que se itera sobre todos los casos para guardar todos los modelos de una sola ejecución, primero se plantean las funciones que definen las restricciones.

In [ ]:
# Función objetivo
def objective_rule(mod):
    # Se asegura de que el costo no sea infinito al calcular el objetivo
    cost = sum(mod.c[u, v] * mod.x[u, v, k]
               for k in mod.K for u in mod.N for v in mod.N
               if u != v and mod.c[u, v] != float('inf'))
    # Añador penalidad por usar arcos infinitos
    penalty = sum(1e9 * mod.x[u, v, k]
                   for k in mod.K for u in mod.N for v in mod.N
                   if u != v and mod.c[u, v] == float('inf'))
    return cost + penalty
   
# Restricción 1: cubrimiento de clientes
def customer_coverage_rule(mod, j):
    return sum(mod.x[u, j, k] for k in mod.K for u in mod.N if u != j) == 1


# Restricción 2a: conservacion de flujo en los nodos de clientes
def flow_conservation_customer_rule(mod, j, k):
    in_flow  = sum(mod.x[u, j, k] for u in mod.N if u != j)
    out_flow = sum(mod.x[j, v, k] for v in mod.N if v != j)
    return in_flow == out_flow

# Restricción 2b: vehiculo sale del deposito asignado
def vehicle_departure_rule(mod, i, k):
    return sum(mod.x[i, v, k] for v in mod.N if v != i) == mod.y[i, k]

# Restricción 2c: vehiculo regresa al deposito asignado
def vehicle_return_rule(mod, i, k):
    return sum(mod.x[u, i, k] for u in mod.N if u != i) == mod.y[i, k]

# Restricción 2d: restricción de asignación de vehículos
def vehicle_assignment_rule(mod, k):
    return sum(mod.y[i, k] for i in mod.I) <= 1

# Restricción 3: capacidad del vehículo
def vehicle_capacity_rule(mod, k):
    demand_served_by_k = sum(
        mod.D[j] * sum(mod.x[u, j, k] for u in mod.N if u != j)
        for j in mod.J
    )
    return demand_served_by_k <= mod.Q[k]

# Restricción 4: rango del vehículo (autonomía)
def vehicle_range_rule(mod, k):
    dist_traveled = sum(
        mod.d[u, v] * mod.x[u, v, k]
        for u in mod.N for v in mod.N
        if u != v and mod.d[u, v] != float('inf')
    )
    infeasible_arc_used = sum(
        mod.x[u, v, k]
        for u in mod.N for v in mod.N
        if u != v and mod.d[u, v] == float('inf')
    )
    # Si se usa un arco infinito, la restricción es infactible
    if pyo.value(infeasible_arc_used) > 0.1:
        return pyo.Constraint.Infeasible
    return dist_traveled <= mod.R[k]


# Restricción 5: capacidad del centro de distribución 
def cd_capacity_rule(mod, i):
    # Suma la demanda D[j] para todos los clientes j que son visitados por un vehículo k que esta asignando al depósito i (y[i,k] = 1).
    # La visita se determina por la suma de arcos entrantes a j para el vehículo k.
    demand_served_from_depot_i = sum(
        mod.D[j] * sum(mod.x[u, j, k] for u in mod.N if u != j)
        for j in mod.J for k in mod.K if mod.y[i, k].is_fixed() and mod.y[i, k].value > 0.5 or not mod.y[i, k].is_fixed() # Incluir k si y[i,k] es 1
    )

    demand_served_from_depot_i_correct = sum(
        mod.D[j] * mod.x[u, j, k]
        for k in mod.K if (mod.y[i,k].value is not None and mod.y[i,k].value > 0.5) # Solo vehículos asignados a i
        for j in mod.J # Para cada cliente
        for u in mod.N if u != j # Suma los arcos entrantes a j por ese vehiculo k
    )
    # y[i,k] es binario para mejor entendimiento del solver
    demand_served_from_depot_i_alt = sum(
         mod.D[j] * mod.x[u,j,k] * mod.y[i,k]
         for k in mod.K for j in mod.J for u in mod.N if u!=j
    )
    return sum(mod.D[j] * mod.x[u, j, k] * mod.y[i, k]
               for j in mod.J for u in mod.N if u != j for k in mod.K
              ) <= mod.A[i]

# Restricción 6: eliminar sub rutas (MTZ)
def subtour_elimination_rule(mod, j, j_prime, k):
    if j == j_prime:
        return pyo.Constraint.Skip
    if mod.d[j, j_prime] == float('inf'):
        return mod.x[j, j_prime, k] == 0
    return (mod.u[j, k] - mod.u[j_prime, k] + mod.n_cust * mod.x[j, j_prime, k] <= mod.n_cust - 1)

# Restricción 7: prevenir loops triviales 
def no_trivial_loops_rule(mod, u, k):
    return mod.x[u, u, k] == 0

# Restricción 8: limitar el número de vehículos al 80% de la capacidad total
def vehicle_limit_rule(mod):
    return sum(mod.y[i, k] for i in mod.I for k in mod.K) <= mod.vehicle_limit_value

In [ ]:
modelos_pyomo = {}

for case in range(1, 3):

    # Cargar los datos del caso específico
    set_I_data = globals()[f'set_I_data{case}']
    set_J_data = globals()[f'set_J_data{case}']
    set_K_data = globals()[f'set_K_data{case}']
    set_N_data = globals()[f'set_N_data{case}']

    param_A_data = globals()[f'param_A_data{case}']
    param_D_data = globals()[f'param_D_data{case}']
    param_Q_data = globals()[f'param_Q_data{case}']
    param_R_data = globals()[f'param_R_data{case}']
    param_n_cust_data = globals()[f'param_n_cust_data{case}']

    param_d_data = globals()[f'param_d_data{case}']
    param_c_data = globals()[f'param_c_data{case}']
    
    # Creación del modelo
    print(f"-----Creando Modelo para el Caso {case} ------")
    model = pyo.ConcreteModel(name="LogistiCo_VRP_CSV_OSRM_UserFunc")

    # Conjuntos
    model.I = pyo.Set(initialize=set_I_data, doc="Distribution Centers")
    model.J = pyo.Set(initialize=set_J_data, doc="Customers")
    model.K = pyo.Set(initialize=set_K_data, doc="Vehicles")
    model.N = pyo.Set(initialize=set_N_data, doc="All Nodes")

    model.vehicle_limit_value = pyo.Param(initialize=int(1 * len(set_K_data)), within=pyo.NonNegativeIntegers, mutable=False)

    
    # Parámetros
    model.A = pyo.Param(model.I, initialize=param_A_data, default=float('inf'), doc="Capacity of CD i")
    model.D = pyo.Param(model.J, initialize=param_D_data, doc="Demand of customer j")
    model.Q = pyo.Param(model.K, initialize=param_Q_data, doc="Capacity of vehicle k")
    model.R = pyo.Param(model.K, initialize=param_R_data, doc="Range of vehicle k")

    # Inicializar desde lo obtenido con OSRM
    model.d = pyo.Param(model.N, model.N, initialize=param_d_data, default=float('inf'), doc="Distance from u to v (km)")
    model.c = pyo.Param(model.N, model.N, initialize=param_c_data, default=float('inf'), doc="Cost from u to v (COP)")
    model.n_cust = pyo.Param(initialize=param_n_cust_data, doc="Number of customers")

    
    # Variables de Decisión
    model.x = pyo.Var(model.N, model.N, model.K, within=pyo.Binary, doc="Vehicle k travels from u to v")
    model.y = pyo.Var(model.I, model.K, within=pyo.Binary, doc="Vehicle k assigned to CD i")
    model.u = pyo.Var(model.J, model.K, within=pyo.NonNegativeReals, bounds=(0, model.n_cust), doc="MTZ auxiliary variable")

    # Preprocesamiento paraarreglar arcos imposibles
    fixed_arcs_count = 0
    for k in model.K:
        assigned_depot = None
        # Encuentra el depot asignado a k (si ya está pre-asignado, si no, esto es más complejo)
        # Por ahora, asumimos que 'y' es variable, así que no podemos saber el depot exacto aún.
        # Por simplificación fijamos arcos si la distancia u->v por si sola excede el rango
        for u in model.N:
            for v in model.N:
                if u == v: continue # Ya cubierto por Constraint 7
                # Si la distancia directa excede el rango O es infinita
                if model.d[u, v] > model.R[k] or model.d[u,v] == float('inf'):
                    if not model.x[u, v, k].is_fixed():
                        model.x[u, v, k].fix(0)
                        fixed_arcs_count += 1
    print(f"Se arreglaron {fixed_arcs_count} x[u,v,k] variables a 0 con base en el rango/infinito")


    # Agregar todo al modelo correspondiente
    model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize, doc="Minimize total cost")
    model.customer_coverage = pyo.Constraint(model.J, rule=customer_coverage_rule, doc="Each customer visited exactly once")
    model.flow_conservation_customer = pyo.Constraint(model.J, model.K, rule=flow_conservation_customer_rule, doc="Flow conservation at customer nodes")
    model.vehicle_departure = pyo.Constraint(model.I, model.K, rule=vehicle_departure_rule, doc="Vehicle leaves assigned depot")
    model.vehicle_return = pyo.Constraint(model.I, model.K, rule=vehicle_return_rule, doc="Vehicle returns to assigned depot")
    model.vehicle_assignment = pyo.Constraint(model.K, rule=vehicle_assignment_rule, doc="Each vehicle assigned to at most one depot")
    model.vehicle_capacity = pyo.Constraint(model.K, rule=vehicle_capacity_rule, doc="Vehicle capacity constraint")
    model.vehicle_range = pyo.Constraint(model.K, rule=vehicle_range_rule, doc="Vehicle range constraint")
    model.cd_capacity = pyo.Constraint(model.I, rule=cd_capacity_rule, doc="CD capacity constraint (Corrected)")
    model.subtour_elimination = pyo.Constraint(model.J, model.J, model.K, rule=subtour_elimination_rule, doc="MTZ subtour elimination")
    model.no_trivial_loops = pyo.Constraint(model.N, model.K, rule=no_trivial_loops_rule, doc="Prevent travel from a node to itself")
    model.vehicle_limit = pyo.Constraint(rule=vehicle_limit_rule, doc="Limit usage to 80% of total vehicles")
    
    print("Estructura del modelo definida usando datos de CSVs y API de OSRM")
    # Almacenar el modelo en estructura de variables globales
    modelos_pyomo[f'case{case}'] = model
    print(f"---- Modelo para el Caso {case} creado correctamente! ----\n\n")

-----Creando Modelo para el Caso 1 ------
Se arreglaron 0 x[u,v,k] variables a 0 con base en el rango/infinito
Estructura del modelo definida usando datos de CSVs y API de OSRM
---- Modelo para el Caso 1 creado correctamente! ----


-----Creando Modelo para el Caso 2 ------
Se arreglaron 581 x[u,v,k] variables a 0 con base en el rango/infinito
Estructura del modelo definida usando datos de CSVs y API de OSRM
---- Modelo para el Caso 2 creado correctamente! ----




### 2.2 Ejecución del Solver

Se utiliza el solver `gurobi` incorporado dentro de la lógica de Pyomo para poder resolver el modelo y encontrar un óptimo, o por lo menos, una solución factible. 

In [120]:
from pyomo.opt import SolverStatus, TerminationCondition

# --- Configurar Solver ---
solver_name = 'gurobi'
try:
    solver = pyo.SolverFactory(solver_name)
    if not solver.available():
        raise RuntimeError(f"Solver '{solver_name}' no encontrado o no ejecutable.")
except Exception as e:
    print(f"ERROR: No se pudo inicializar el solver '{solver_name}'. Error: {e}", file=sys.stderr)
    sys.exit(1)

In [121]:
# Resolver el modelo con timeout diferente por caso
for case in range(1, 3):
    print(f"\n\n======= RESOLVIENDO CASO {case} =========")
    model = modelos_pyomo[f'case{case}']

    # Resolver el modelo
    print("Resolviendo...")

    if case == 1:
        solver.options['TimeLimit'] = 900
    elif case == 2:
        solver.options['TimeLimit'] = 900

    results = solver.solve(model, tee=True)

    # Verificar resultado del Solver
    print("\n--- Resultado del Solver ---")
    print(results)

    status = results.solver.status
    termination = results.solver.termination_condition

    # Condiciones aceptables: solución óptima, factible, o parada por tiempo con solución
    accepted = (
        status in {SolverStatus.ok, SolverStatus.aborted}
        and termination in {
            TerminationCondition.optimal,
            TerminationCondition.feasible,
            TerminationCondition.maxTimeLimit,
        }
    )

    if accepted:
        mensajes = {
            TerminationCondition.optimal: "\n Solución Óptima Encontrada",
            TerminationCondition.maxTimeLimit: "\n Límite de Tiempo Alcanzado - Solución Factible ",
            TerminationCondition.feasible: "\n Solución Factible Encontrada ",
        }
        print(mensajes.get(termination, "\n Solución Aceptada"))
        print(f"Costo total mínimo: {pyo.value(model.objective):,.2f} COP")

        # Asignación de vehículos
        print("\n--- Asignaciones de Vehículos (y_ik) ---")
        assignments = {}
        assigned_vehicles = 0
        for k in model.K:
            assigned = next((i for i in model.I if pyo.value(model.y[i, k], exception=False) > 0.5), None)
            if assigned:
                print(f"Vehículo {k} parte del CD {assigned}")
                assignments[k] = assigned
                assigned_vehicles += 1
            else:
                print(f"Vehículo {k} no se usa.")
                assignments[k] = 'Unassigned'

        if assigned_vehicles == 0:
            print("Ningún vehículo fue asignado.")

        # Rutas
        print("\n--- Rutas (x_uvk) ---")
        total_distance = total_demand = 0
        vehicles_used = set()

        for k in model.K:
            start = assignments[k]
            if start == 'Unassigned':
                continue

            route = [start]
            current = start
            dist = demand = 0
            visited = {start}
            route_found = False

            for _ in range(len(model.N) + 2):
                next_nodes = [
                    v for v in model.N if v != current and pyo.value(model.x[current, v, k], exception=False) > 0.5
                ]
                if len(next_nodes) > 1:
                    print(f"ERROR: Múltiples opciones desde {current} para el vehículo {k}: {next_nodes}")
                    route = [start, "Error"]
                    break
                elif not next_nodes:
                    if current != start:
                        print(f"  ERROR: Ruta incompleta desde {current} para el vehículo {k}.")
                        route.append("Error")
                    break

                next_node = next_nodes[0]
                arc_dist = model.d[current, next_node]
                if arc_dist == float('inf'):
                    print(f"  ERROR: Arco no permitido de {current} a {next_node}")
                    route.append(f"{next_node}(inf)")
                    break

                dist += arc_dist
                route.append(next_node)
                visited.add(next_node)
                if next_node in model.J:
                    demand += model.D[next_node]
                current = next_node

                if current == start:
                    route_found = True
                    break

            print(f"\nRuta del Vehículo {k} (desde {start}):")
            print(f"  {' hasta '.join(route)}")
            if route_found:
                print(f"  Distancia: {dist:.2f} km (Límite: {model.R[k]:.2f})")
                print(f"  Demanda: {demand:.2f} kg (Capacidad: {model.Q[k]:.2f})")
                total_distance += dist
                total_demand += demand
                vehicles_used.add(k)
                if dist > model.R[k] + 1e-6:
                    print(f"Vehículo {k} excede el rango permitido.")
                if demand > model.Q[k] + 1e-6:
                    print(f"Vehículo {k} excede la capacidad.")
            else:
                print("Ruta incompleta o errónea.")

        # Resumen final de los resultados
        print("\n--- Resumen General ---")
        print(f"Distancia total recorrida: {total_distance:.2f} km")
        print(f"Demanda total atendida: {total_demand:.2f} kg")
        total_expected = sum(param_D_data.values())
        print(f"Demanda total esperada: {total_expected:.2f} kg")
        print(f"Número de vehículos utilizados: {len(vehicles_used)} de {len(model.K)}")
        if abs(total_demand - total_expected) > 1e-6:
            print("La demanda servida no coincide con la esperada.")

    elif termination == TerminationCondition.infeasible:
        print("\n--- El Modelo es Inviable ---")
        print("No existe una solución que cumpla todas las restricciones.")

    else:
        print("\n--- No se encontró solución óptima o factible ---")
        print(f"Estado del Solver: {status}")
        print(f"Condición de Terminación: {termination}")



======= RESOLVIENDO CASO 1 =========
Resolviendo...
Set parameter Username
Set parameter LicenseID to value 2656939
Academic license - for non-commercial use only - expires 2026-04-25
Read LP format model from file C:\Users\nicol\AppData\Local\Temp\tmpcmi5d3lv.pyomo.lp
Reading time = 0.39 seconds
x15601: 14570 rows, 15601 columns, 111289 nonzeros
Set parameter TimeLimit to value 900
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  900

Optimize a model with 14570 rows, 15601 columns and 111289 nonzeros
Model fingerprint: 0xd2f939a5
Variable types: 577 continuous, 15024 integer (15024 binary)
Coefficient statistics:
  Matrix range     [7e-01, 3e+01]
  Objective range  [1e+04, 6e+05]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+02]
Fou

### 2.3 Extensión para tratabilidad del Caso 3

#### 2.3.1 Estrategia de División del Problema

A medida que se asignan los

In [107]:

# Heurística mejorada de asignación (cliente, vehículo) → depósito
def heuristica_asignacion_mejorada(set_I, set_J, set_K, param_A, param_D, param_Q, param_R, param_d, param_c):
    """
    Heurística para asignar clientes y vehículos a depósitos considerando:
    - Capacidad de los depósitos
    - Capacidad de carga de los vehículos
    - Autonomía de los vehículos
    - Distancia entre clientes y depósitos
    
    Devuelve:
    - z: Diccionario de asignación cliente → depósito
    - w: Diccionario de asignación cliente → vehículo
    - depot_for_veh: Diccionario de asignación vehículo → depósito
    """
    # 1) Capacidades remanentes de depósitos
    cap_rem = {i: param_A[i] for i in set_I}
    
    # 2) Seguimiento vehículo → depósito, y sus capacidades
    depot_for_veh = {k: None for k in set_K}
    cap_rem_veh = {k: param_Q[k] for k in set_K}
    
    # 3) Autonomía remanente de vehículos
    autonomia_rem = {k: param_R[k] for k in set_K}
    
    # 4) Asignaciones iniciales
    z = {}  # cliente→depósito
    w = {}  # cliente→vehículo
    
    # 5) Pre-procesamiento: Calcular distancias totales estimadas para cada cliente
    # Esto nos da una idea de qué tan "costoso" es servir a cada cliente
    dificultad_cliente = {}
    for j in set_J:
        # Promedio de distancia a los depósitos accesibles
        depots_accesibles = [i for i in set_I if param_d[i, j] != float('inf')]
        if not depots_accesibles:
            # Encontrar el depósito más cercano (con distancia finita mínima)
            min_dist = float('inf')
            closest_depot = None
            for i in set_I:
                if param_d[i, j] < min_dist:
                    min_dist = param_d[i, j]
                    closest_depot = i
            depots_accesibles = [closest_depot]
        
        # La dificultad se basa en la distancia y la demanda
        dificultad_cliente[j] = sum(param_d[i, j] for i in depots_accesibles) / len(depots_accesibles)
        dificultad_cliente[j] *= param_D[j]  # Ponderar por demanda
    
    # 6) Ordenar clientes por dificultad decreciente (atender primero los más difíciles)
    clientes_ordenados = sorted(set_J, key=lambda j: -dificultad_cliente[j])
    
    # 7) Bucle greedy principal
    for j in clientes_ordenados:
        mejor_costo = float('inf')
        mejor_i = None
        mejor_k = None
        
        # 7.1) Probar todas las combinaciones factibles de depósito y vehículo
        for i in set_I:
            # Verificar si el depósito tiene capacidad
            if cap_rem[i] < param_D[j] or param_d[i, j] == float('inf'):
                continue
                
            # Buscar vehículos que podrían servir desde este depósito
            for k in set_K:
                # Verificar si el vehículo tiene capacidad
                if cap_rem_veh[k] < param_D[j]:
                    continue
                    
                # Si el vehículo ya está asignado a otro depósito, saltar
                if depot_for_veh[k] is not None and depot_for_veh[k] != i:
                    continue
                    
                # Estimar costo de la ruta: i -> j -> i (ida y vuelta al depósito)
                dist_estimada = param_d[i, j] + param_d[j, i]
                
                # Verificar autonomía
                if dist_estimada > autonomia_rem[k]:
                    continue
                    
                # Calcular costo total
                costo_estimado = param_c[i, j] + param_c[j, i]
                
                # Actualizar mejor opción
                if costo_estimado < mejor_costo:
                    mejor_costo = costo_estimado
                    mejor_i = i
                    mejor_k = k
        
        # 7.2) Asignar la mejor combinación encontrada
        if mejor_i is None:
            # Intentar reasignar vehículos si no encontramos solución
            reasignacion_exitosa = False
            
            # Primera opción: buscar vehículos sin asignar
            for i in sorted(set_I, key=lambda i: param_d[i, j] if param_d[i, j] != float('inf') else float('inf')):
                if cap_rem[i] < param_D[j] or param_d[i, j] == float('inf'):
                    continue
                    
                # Buscar vehículos sin asignar
                for k in [k for k in set_K if depot_for_veh[k] is None]:
                    dist_estimada = param_d[i, j] + param_d[j, i]
                    if param_D[j] <= param_Q[k] and dist_estimada <= param_R[k]:
                        mejor_i = i
                        mejor_k = k
                        reasignacion_exitosa = True
                        break
                
                if reasignacion_exitosa:
                    break
            
            # Segunda opción: permitir sobrecarga, priorizando vehículos menos sobrecargados
            if not reasignacion_exitosa:                
                # Encontrar el depósito más cercano
                depots_accesibles = [(i, param_d[i, j]) for i in set_I if param_d[i, j] != float('inf')]
                depots_accesibles.sort(key=lambda x: x[1])  # Ordenar por distancia
                
                # Para cada depósito accesible, buscar el vehículo menos sobrecargado
                for i, _ in depots_accesibles:
                    # Verificar que el depósito tenga capacidad
                    if cap_rem[i] < param_D[j]:
                        continue
                        
                    # Buscar vehículos ya asignados a este depósito
                    veh_depot = [k for k, depot in depot_for_veh.items() if depot == i]
                    if not veh_depot:
                        # Si no hay vehículos asignados, buscar uno libre
                        veh_libres = [k for k, depot in depot_for_veh.items() if depot is None]
                        if veh_libres:
                            mejor_i = i
                            mejor_k = max(veh_libres, key=lambda k: param_Q[k])
                            reasignacion_exitosa = True
                            break
                    else:
                        # Elegir el vehículo con mayor capacidad restante
                        mejor_i = i
                        mejor_k = max(veh_depot, key=lambda k: cap_rem_veh[k])
                        reasignacion_exitosa = True
                        break
                
                if not reasignacion_exitosa:
                    # Última opción: elegir cualquier depósito con capacidad y asignar el vehículo con más capacidad
                    for i in sorted(set_I, key=lambda i: param_d[i, j] if param_d[i, j] != float('inf') else float('inf')):
                        if cap_rem[i] < param_D[j] or param_d[i, j] == float('inf'):
                            continue
                            
                        mejor_i = i
                        mejor_k = max(set_K, key=lambda k: cap_rem_veh[k])
                        depot_for_veh[mejor_k] = i  # Forzar asignación al depósito
                        reasignacion_exitosa = True
                        break

        
        # 7.3) Registrar asignación
        z[j] = mejor_i
        w[j] = mejor_k
        
        # 7.4) Actualizar recursos disponibles
        cap_rem[mejor_i] -= param_D[j]
        cap_rem_veh[mejor_k] -= param_D[j]
        autonomia_rem[mejor_k] -= (param_d[mejor_i, j] + param_d[j, mejor_i])
        
        # 7.5) Si el vehículo no tenía depósito asignado, asignarlo
        if depot_for_veh[mejor_k] is None:
            depot_for_veh[mejor_k] = mejor_i
    
    # 8) Optimización post-procesamiento para subproblemas pequeños
    for i in set_I:
        clientes_i = [j for j in set_J if z[j] == i]
        if 1 <= len(clientes_i) <= 4:
            optimizar_subproblema_pequeno(i, clientes_i, z, w, depot_for_veh, cap_rem_veh, 
                                         autonomia_rem, param_D, param_d, set_K)
    
    return z, w, depot_for_veh



def optimizar_subproblema_pequeno(i, clientes_i, z, w, depot_for_veh, cap_rem_veh, 
                                autonomia_rem, param_D, param_d, set_K):
    """
    Optimiza la asignación de vehículos para subproblemas pequeños (≤4 clientes)
    intentando asignar un vehículo distinto a cada cliente si es posible.
    Se preservan siempre las asignaciones cliente→depósito.
    """
    # Verificar si ya tenemos vehículos distintos
    veh_act = [w[j] for j in clientes_i]
    if len(set(veh_act)) == len(clientes_i):
        return  # ya son todos distintos
    
    # No optimizar si hay un solo cliente (ya tiene su asignación)
    if len(clientes_i) == 1:
        return
    
    print(f"Optimizando subproblema para depósito {i} con {len(clientes_i)} clientes...")
    
    # Candidatos: primero los de depot, luego libres
    cand = [k for k, v in depot_for_veh.items() if v == i] + \
           [k for k, v in depot_for_veh.items() if v is None]
    
    # Si no hay suficientes candidatos, no tiene sentido optimizar
    if len(cand) < len(clientes_i):
        print(f"  - No hay suficientes vehículos candidatos ({len(cand)}) para los {len(clientes_i)} clientes")
        return
    
    # Ordenar clientes por demanda decreciente
    clientes_ordenados = sorted(clientes_i, key=lambda j: -param_D[j])
    
    # Crear copia de estructuras para simular cambios
    w_temp = w.copy()
    cap_rem_veh_temp = cap_rem_veh.copy()
    autonomia_rem_temp = autonomia_rem.copy()
    depot_for_veh_temp = depot_for_veh.copy()
    
    # Intentar asignar vehículos distintos
    usados = set()
    for j in clientes_ordenados:
        # Devolver demanda del vehículo anterior
        viejo = w_temp[j]
        cap_rem_veh_temp[viejo] += param_D[j]
        # Estimación simple ida y vuelta
        if autonomia_rem_temp[viejo] != float('inf'):  # Evitar error con infinito
            autonomia_rem_temp[viejo] += (param_d[i, j] + param_d[j, i])
        
        # Asignar primer candidato no usado y factible
        asignado = False
        for k in cand:
            if k not in usados:
                # Verificar capacidad y autonomía
                autonomia_suficiente = (autonomia_rem_temp[k] == float('inf')) or \
                                      (autonomia_rem_temp[k] >= (param_d[i, j] + param_d[j, i]))
                                      
                if cap_rem_veh_temp[k] >= param_D[j] and autonomia_suficiente:
                    w_temp[j] = k
                    usados.add(k)
                    cap_rem_veh_temp[k] -= param_D[j]
                    
                    if autonomia_rem_temp[k] != float('inf'):
                        autonomia_rem_temp[k] -= (param_d[i, j] + param_d[j, i])
                    
                    if depot_for_veh_temp[k] is None:
                        depot_for_veh_temp[k] = i
                    
                    asignado = True
                    break
        
        if not asignado:
            # Si no hay candidato factible, dejar el vehículo original
            w_temp[j] = viejo
            cap_rem_veh_temp[viejo] -= param_D[j]
            
            if autonomia_rem_temp[viejo] != float('inf'):
                autonomia_rem_temp[viejo] -= (param_d[i, j] + param_d[j, i])
    
    # Verificar si la nueva solución tiene más vehículos distintos
    new_veh_act = [w_temp[j] for j in clientes_i]
    new_veh_unicos = len(set(new_veh_act))
    veh_unicos_actuales = len(set(veh_act))
    
    if new_veh_unicos > veh_unicos_actuales:
        print(f"  - Mejora: de {veh_unicos_actuales} a {new_veh_unicos} vehículos distintos")
        # Actualizar solución real
        for j in clientes_i:
            w[j] = w_temp[j]
        
        # Actualizar recursos
        for k in set_K:
            cap_rem_veh[k] = cap_rem_veh_temp[k]
            autonomia_rem[k] = autonomia_rem_temp[k]
            depot_for_veh[k] = depot_for_veh_temp[k]
    else:
        print(f"  - Sin mejora: se mantienen {veh_unicos_actuales} vehículos distintos")


In [108]:
# =============================================================================
# 1) DESCOMPOSICIÓN POR DEPÓSITO
# =============================================================================
case = 3
# datos originales
set_I = globals()[f'set_I_data{case}']
set_J = globals()[f'set_J_data{case}']
set_K = globals()[f'set_K_data{case}']
set_N = globals()[f'set_N_data{case}']

param_A = globals()[f'param_A_data{case}']
param_D = globals()[f'param_D_data{case}']
param_Q = globals()[f'param_Q_data{case}']
param_R = globals()[f'param_R_data{case}']
param_n_cust = globals()[f'param_n_cust_data{case}']
param_d = globals()[f'param_d_data{case}']
param_c = globals()[f'param_c_data{case}']

# Heurística greedy de asignación (cliente, vehículo) → depósito
z, w, depot_for_veh = heuristica_asignacion_mejorada(
    set_I, set_J, set_K, param_A, param_D, param_Q, param_R, param_d, param_c
)

# Generación de subproblemas
subs_md = []
for i in set_I:
    clientes_i = [j for j in set_J if z[j] == i]
    if not clientes_i: continue
    vehiculos_i = sorted({ w[j] for j in clientes_i })
    nodos_i = [i] + clientes_i
    subs_md.append({
        'I': [i],
        'J': clientes_i,
        'K': vehiculos_i,
        'N': nodos_i,
        'A': {i: param_A[i]},
        'D': {j: param_D[j] for j in clientes_i},
        'Q': {k: param_Q[k] for k in vehiculos_i},
        'R': {k: param_R[k] for k in vehiculos_i},
        'n_cust': len(clientes_i),
        'd': {(u, v): param_d[u, v] for u in nodos_i for v in nodos_i},
        'c': {(u, v): param_c[u, v] for u in nodos_i for v in nodos_i},
    })

# Construcción de modelos
modelos_pyomo_case3_multidepot = {}
for idx, sub in enumerate(subs_md, start=1):
    m = pyo.ConcreteModel(f"MD3_{idx}")
    m.I = pyo.Set(initialize=sub['I'])
    m.J = pyo.Set(initialize=sub['J'])
    m.K = pyo.Set(initialize=sub['K'])
    m.N = pyo.Set(initialize=sub['N'])
    m.A = pyo.Param(m.I, initialize=sub['A'])
    m.D = pyo.Param(m.J, initialize=sub['D'])
    m.Q = pyo.Param(m.K, initialize=sub['Q'])
    m.R = pyo.Param(m.K, initialize=sub['R'])
    m.n_cust = pyo.Param(initialize=sub['n_cust'])
    m.d = pyo.Param(m.N, m.N, initialize=sub['d'], default=float('inf'))
    m.c = pyo.Param(m.N, m.N, initialize=sub['c'], default=float('inf'))
    m.x = pyo.Var(m.N, m.N, m.K, domain=pyo.Binary)
    m.y = pyo.Var(m.I, m.K, domain=pyo.Binary)
    m.u = pyo.Var(m.J, m.K, bounds=(0, sub['n_cust']), domain=pyo.NonNegativeReals)

    # Agregar restricciones
    m.customer_coverage       = pyo.Constraint(m.J, rule=customer_coverage_rule)
    m.flow_conservation       = pyo.Constraint(m.J, m.K, rule=flow_conservation_customer_rule)
    m.vehicle_departure       = pyo.Constraint(m.I, m.K, rule=vehicle_departure_rule)
    m.vehicle_return          = pyo.Constraint(m.I, m.K, rule=vehicle_return_rule)
    m.vehicle_assignment      = pyo.Constraint(m.K, rule=vehicle_assignment_rule)
    m.vehicle_capacity        = pyo.Constraint(m.K, rule=vehicle_capacity_rule)
    m.vehicle_range           = pyo.Constraint(m.K, rule=vehicle_range_rule)
    m.cd_capacity             = pyo.Constraint(m.I, rule=cd_capacity_rule)
    m.subtour_elimination     = pyo.Constraint(m.J, m.J, m.K, rule=subtour_elimination_rule)
    m.no_trivial_loops        = pyo.Constraint(m.N, m.K, rule=no_trivial_loops_rule)

    # Objective
    m.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    modelos_pyomo_case3_multidepot[f"3_{idx}"] = m


Optimizando subproblema para depósito D2 con 4 clientes...
  - Mejora: de 1 a 2 vehículos distintos
Optimizando subproblema para depósito D8 con 2 clientes...
  - Sin mejora: se mantienen 1 vehículos distintos
Optimizando subproblema para depósito D9 con 3 clientes...
  - Mejora: de 1 a 3 vehículos distintos
Optimizando subproblema para depósito D10 con 4 clientes...
  - No hay suficientes vehículos candidatos (1) para los 4 clientes
Optimizando subproblema para depósito D11 con 4 clientes...
  - No hay suficientes vehículos candidatos (1) para los 4 clientes


In [109]:
for idx, sub in enumerate(subs_md, start=1):
    deposito   = sub['I'][0]
    clientes   = sub['J']
    vehiculos  = sub['K']

    print(f"--- Subproblema {idx} ---")
    print(f"Depósito asignado: {deposito}")
    print(f"Clientes     : {', '.join(map(str, clientes))}")
    print(f"Vehículos    : {', '.join(map(str, vehiculos))}")
    print()

--- Subproblema 1 ---
Depósito asignado: D2
Clientes     : C53, C62, C78, C83
Vehículos    : V26, V8

--- Subproblema 2 ---
Depósito asignado: D3
Clientes     : C5, C6, C31, C40, C67, C77
Vehículos    : V1

--- Subproblema 3 ---
Depósito asignado: D4
Clientes     : C2, C4, C47, C58, C63, C66
Vehículos    : V4

--- Subproblema 4 ---
Depósito asignado: D5
Clientes     : C3, C16, C17, C19, C33, C42, C46, C51, C60, C64, C65, C76
Vehículos    : V12, V5

--- Subproblema 5 ---
Depósito asignado: D6
Clientes     : C1, C11, C26, C29, C32, C55, C74, C75
Vehículos    : V14, V7

--- Subproblema 6 ---
Depósito asignado: D7
Clientes     : C10, C12, C14, C15, C20, C23, C24, C28, C30, C34, C35, C38, C39, C43, C44, C48, C50, C56, C57, C59, C69, C72, C73, C84, C85, C87, C89, C90
Vehículos    : V13, V16, V17, V18, V19, V20, V21, V22, V23, V24, V25, V27, V28, V9

--- Subproblema 7 ---
Depósito asignado: D8
Clientes     : C9, C81
Vehículos    : V10

--- Subproblema 8 ---
Depósito asignado: D9
Clientes     

#### 2.3.2 Solución de los Subproblemas

In [110]:
from pyomo.opt import SolverStatus, TerminationCondition

# --- Configurar Solver ---
solver_name = 'gurobi'
try:
    solver = pyo.SolverFactory(solver_name)
    solver.options['TimeLimit'] = 300  # 5 minutos por subproblema
    if not solver.available():
        raise RuntimeError(f"Solver '{solver_name}' no encontrado o no ejecutable.")
except Exception as e:
    print(f"ERROR: No se pudo inicializar el solver '{solver_name}'. Error: {e}", file=sys.stderr)
    sys.exit(1)

In [111]:
modelos_pyomo_case3 = []
# Resolver el modelo con timeout diferente por caso
for name, mod in modelos_pyomo_case3_multidepot.items():
    print(f"\n\n======= RESOLVIENDO SUBPROBLEMA {name} DEL CASO 3 =========")
    model = mod

    # Resolver el modelo
    print("Resolviendo...")

    results = solver.solve(model, tee=True)

    # Verificar resultado del Solver
    print("\n--- Resultado del Solver ---")
    print(results)
    modelos_pyomo_case3.append(model)

    status = results.solver.status
    termination = results.solver.termination_condition

    # Condiciones aceptables: solución óptima, factible, o parada por tiempo con solución
    accepted = (
        status in {SolverStatus.ok, SolverStatus.aborted}
        and termination in {
            TerminationCondition.optimal,
            TerminationCondition.feasible,
            TerminationCondition.maxTimeLimit,
        }
    )

    if accepted:
        mensajes = {
            TerminationCondition.optimal: "\n Solución Óptima Encontrada",
            TerminationCondition.maxTimeLimit: "\n Límite de Tiempo Alcanzado - Solución Factible ",
            TerminationCondition.feasible: "\n Solución Factible Encontrada ",
        }
        print(mensajes.get(termination, "\n Solución Aceptada"))
        print(f"Costo total mínimo: {pyo.value(model.objective):,.2f} COP")

        # Asignación de vehículos
        print("\n--- Asignaciones de Vehículos (y_ik) ---")
        assignments = {}
        assigned_vehicles = 0
        for k in model.K:
            assigned = next((i for i in model.I if pyo.value(model.y[i, k], exception=False) > 0.5), None)
            if assigned:
                print(f"Vehículo {k} parte del CD {assigned}")
                assignments[k] = assigned
                assigned_vehicles += 1
            else:
                print(f"Vehículo {k} no se usa.")
                assignments[k] = 'Unassigned'

        if assigned_vehicles == 0:
            print("Ningún vehículo fue asignado.")

        # Rutas
        print("\n--- Rutas (x_uvk) ---")
        total_distance = total_demand = 0
        vehicles_used = set()

        for k in model.K:
            start = assignments[k]
            if start == 'Unassigned':
                continue

            route = [start]
            current = start
            dist = demand = 0
            visited = {start}
            route_found = False

            for _ in range(len(model.N) + 2):
                next_nodes = [
                    v for v in model.N if v != current and pyo.value(model.x[current, v, k], exception=False) > 0.5
                ]
                if len(next_nodes) > 1:
                    print(f"ERROR: Múltiples opciones desde {current} para el vehículo {k}: {next_nodes}")
                    route = [start, "Error"]
                    break
                elif not next_nodes:
                    if current != start:
                        print(f"  ERROR: Ruta incompleta desde {current} para el vehículo {k}.")
                        route.append("Error")
                    break

                next_node = next_nodes[0]
                arc_dist = model.d[current, next_node]
                if arc_dist == float('inf'):
                    print(f"  ERROR: Arco no permitido de {current} a {next_node}")
                    route.append(f"{next_node}(inf)")
                    break

                dist += arc_dist
                route.append(next_node)
                visited.add(next_node)
                if next_node in model.J:
                    demand += model.D[next_node]
                current = next_node

                if current == start:
                    route_found = True
                    break

            print(f"\nRuta del Vehículo {k} (desde {start}):")
            print(f"  {' hasta '.join(route)}")
            if route_found:
                print(f"  Distancia: {dist:.2f} km (Límite: {model.R[k]:.2f})")
                print(f"  Demanda: {demand:.2f} kg (Capacidad: {model.Q[k]:.2f})")
                total_distance += dist
                total_demand += demand
                vehicles_used.add(k)
                if dist > model.R[k] + 1e-6:
                    print(f"Vehículo {k} excede el rango permitido.")
                if demand > model.Q[k] + 1e-6:
                    print(f"Vehículo {k} excede la capacidad.")
            else:
                print("Ruta incompleta o errónea.")

        # Resumen final de los resultados
        print("\n--- Resumen General ---")
        print(f"Distancia total recorrida: {total_distance:.2f} km")
        print(f"Demanda total atendida: {total_demand:.2f} kg")
        total_expected = sum(mod.D.values())
        print(f"Demanda total esperada: {total_expected:.2f} kg")
        print(f"Número de vehículos utilizados: {len(vehicles_used)} de {len(model.K)}")
        if abs(total_demand - total_expected) > 1e-6:
            print("La demanda servida no coincide con la esperada.")

    elif termination == TerminationCondition.infeasible:
        print("\n--- El Modelo es Inviable ---")
        print("No existe una solución que cumpla todas las restricciones.")

    else:
        print("\n--- No se encontró solución óptima o factible ---")
        print(f"Estado del Solver: {status}")
        print(f"Condición de Terminación: {termination}")



======= RESOLVIENDO SUBPROBLEMA 3_1 DEL CASO 3 =========
Resolviendo...


Set parameter Username
Set parameter LicenseID to value 2656939
Academic license - for non-commercial use only - expires 2026-04-25
Read LP format model from file C:\Users\nicol\AppData\Local\Temp\tmpexxb5883.pyomo.lp
Reading time = 0.02 seconds
x61: 57 rows, 61 columns, 273 nonzeros
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 57 rows, 61 columns and 273 nonzeros
Model fingerprint: 0xc1851806
Model has 1 quadratic constraint
Variable types: 9 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+01]
  QMatrix range    [1e+01, 1e+01]
  Objective range  [1e+04, 5e+05]
  Bounds range     [1e+00, 4e+00]
  RHS range        [1e+00, 1e+02]
  QRHS range       

#### 2.3.3 Combinación de Soluciones

In [112]:
x_sol = {}
y_sol = {}
objetivo = 0

for model in modelos_pyomo_case3_multidepot.values():
    # Limpiar variables y restricciones del modelo
    x_sol.update({(u, v, k): pyo.value(model.x[u, v, k]) for u in model.N for v in model.N for k in model.K})
    y_sol.update({(i, k): pyo.value(model.y[i, k]) for i in model.I for k in model.K})
    objetivo += pyo.value(model.objective)

# Imprimir sólo las variables no nulas
print("\n--- Variables de Asignación (x_uvk) ---")
for (u, v, k), value in x_sol.items():
    if value > 0.1:
        print(f"x[{u}, {v}, {k}] = {value:.2f}")
print("\n--- Variables de Asignación de Depósito (y_ik) ---")
for (i, k), value in y_sol.items():
    if value > 0.1:
        print(f"y[{i}, {k}] = {value:.2f}")
print(f"\nCosto total mínimo: {objetivo:.2f} COP")
print("\n--- Resumen de Resultados ---")
print(f"Total de depósitos usados: {len(set(i for i, k in y_sol if y_sol[i, k] > 0.1))}")
print(f"Total de vehículos usados: {len(set(k for i, k in y_sol if y_sol[i, k] > 0.1))}")
print(f"Total de clientes atendidos: {len(set(v for (u, v, k), value in x_sol.items() if value > 0.1 and v not in set_I))}")
print(f"Total de clientes no atendidos: {len(set_J) - len(set(v for (u, v, k), value in x_sol.items() if value > 0.1 and v not in set_I))}")
print("\n--- Fin de la Ejecución ---")


--- Variables de Asignación (x_uvk) ---
x[D2, C53, V8] = 1.00
x[C53, C83, V8] = 1.00
x[C62, C78, V8] = 1.00
x[C78, D2, V8] = 1.00
x[C83, C62, V8] = 1.00
x[D3, C31, V1] = 1.00
x[C5, C77, V1] = 1.00
x[C6, C67, V1] = 1.00
x[C31, C5, V1] = 1.00
x[C40, D3, V1] = 1.00
x[C67, C40, V1] = 1.00
x[C77, C6, V1] = 1.00
x[D4, C58, V4] = 1.00
x[C2, C63, V4] = 1.00
x[C4, C47, V4] = 1.00
x[C47, C2, V4] = 1.00
x[C58, C66, V4] = 1.00
x[C63, D4, V4] = 1.00
x[C66, C4, V4] = 1.00
x[D5, C17, V12] = 1.00
x[D5, C51, V5] = 1.00
x[C3, C16, V12] = 1.00
x[C16, D5, V12] = 1.00
x[C17, C3, V12] = 1.00
x[C19, D5, V5] = 1.00
x[C33, C64, V5] = 1.00
x[C42, C33, V5] = 1.00
x[C46, C76, V5] = 1.00
x[C51, C60, V5] = 1.00
x[C60, C46, V5] = 1.00
x[C64, C19, V5] = 1.00
x[C65, C42, V5] = 1.00
x[C76, C65, V5] = 1.00
x[D6, C74, V7] = 1.00
x[C1, C26, V7] = 1.00
x[C11, C75, V7] = 1.00
x[C26, C55, V7] = 1.00
x[C29, C32, V7] = 1.00
x[C32, C11, V7] = 1.00
x[C55, D6, V7] = 1.00
x[C74, C29, V7] = 1.00
x[C75, C1, V7] = 1.00
x[D7, C10, V1

## 3. Reportes y Gráficas

### 3.1 Generación de Archivos y Reportes

Genere un único informe CSV detallado que resuma la ruta de cada vehículo, incluyendo secuencia, clientes, demandas, distancia y costo, coincidiendo con el formato específico solicitado.

In [139]:
import os
import csv
import pyomo.environ as pyo

C_KM = 20700  # COP/km

def collect_vehicle_rows(solution_model):
    """
    Recorre un modelo Pyomo resuelto y devuelve la lista de filas
    (diccionarios) con la estructura esperada por el CSV.
    """
    rows = []
    # Reconstruir asignaciones y rutas (mismo código que ya tienes)
    assignments = {
        k: next((i for i in solution_model.I
                 if pyo.value(solution_model.y[i, k], exception=False) > 0.5),
                None)
        for k in solution_model.K
    }

    for k, start_depot in assignments.items():
        if start_depot is None: 
            continue

        # Verificar que realmente sale del depósito
        if not any(pyo.value(solution_model.x[start_depot, v, k], exception=False) > 0.5
                   for v in solution_model.N if v != start_depot):
            continue

        # Trazado de ruta (idéntico al tuyo)…
        route_nodes = [start_depot]
        visited_clients_ordered = []
        demands_ordered = []
        route_distance = 0.0
        current_node = start_depot
        max_steps = len(solution_model.N) + 2
        route_valid = True

        for step in range(max_steps):
            next_nodes = [
                v for v in solution_model.N
                if v != current_node and
                   pyo.value(solution_model.x[current_node, v, k], exception=False) > 0.5
            ]
            if len(next_nodes) == 0:
                if current_node == start_depot and step > 0:
                    break
                route_valid = False
                break
            next_node = next_nodes[0]

            arc_dist = solution_model.d[current_node, next_node]
            if arc_dist == float('inf'):
                route_valid = False
                break
            route_distance += arc_dist
            route_nodes.append(next_node)
            if next_node in solution_model.J:
                visited_clients_ordered.append(next_node)
                demands_ordered.append(solution_model.D[next_node])
            current_node = next_node
            if current_node == start_depot:
                break
            if step == max_steps - 1:
                route_valid = False

        if not route_valid:
            continue

        # Construir el diccionario de la fila
        row = {
            'VehicleId': k,
            'DepotId': start_depot,
            'InitialLoad': round(sum(demands_ordered), 2),
            'RouteSequence': ' - '.join(map(str, route_nodes)),
            'ClientsServed': len(set(visited_clients_ordered)),
            'DemandsSatisfied': ' - '.join(
                str(int(d)) if d == int(d) else str(d)
                for d in demands_ordered
            ),
            'TotalDistance': round(route_distance, 2),
            'TotalTime': round(route_distance * 50 / 60, 2),  # Asumiendo 50 km/h
            'FuelCost': round(route_distance * C_KM),
        }
        rows.append(row)
    return rows

def generate_detailed_vehicle_report_multiple(models_list, case_number):
    """
    Genera UN SOLO CSV detallado combinando la salida de varios modelos Pyomo.
    """
    group_name = os.getenv('GROUP_NAME', 'Grupo10')

    output_dir = f"{group_name}-caso-{case_number}-results"
    os.makedirs(output_dir, exist_ok=True)
    report_filename = os.path.join(
        output_dir,
        f"{group_name}-caso-{case_number}-ReporteDetallado.csv"
    )

    # Recolectar filas de todos los modelos
    all_rows = []
    for model in models_list:
        all_rows.extend(collect_vehicle_rows(model))

    if not all_rows:
        print("- No valid vehicle routes found in any submodelo. CSV no generado.")
        return

    # Definir cabecera
    headers = [
        'VehicleId', 'DepotId', 'InitialLoad', 'RouteSequence',
        'ClientsServed', 'DemandsSatisfied', 'TotalDistance',
        'TotalTime', 'FuelCost'
    ]

    # Escribir una sola vez
    with open(report_filename, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        writer.writeheader()
        writer.writerows(all_rows)

    print(f"- Detailed vehicle report created successfully: {report_filename}")


generate_detailed_vehicle_report_multiple([modelos_pyomo['case1']], 1)
generate_detailed_vehicle_report_multiple([modelos_pyomo['case2']], 2)
generate_detailed_vehicle_report_multiple(modelos_pyomo_case3, 3)


- Detailed vehicle report created successfully: Grupo10-caso-1-results\Grupo10-caso-1-ReporteDetallado.csv
- Detailed vehicle report created successfully: Grupo10-caso-2-results\Grupo10-caso-2-ReporteDetallado.csv
- Detailed vehicle report created successfully: Grupo10-caso-3-results\Grupo10-caso-3-ReporteDetallado.csv


### 3.2 Visualización con Mapas

In [133]:
import os
import pyomo.environ as pyo
import requests # Para llamadas API OSRM
import time     # Para delays entre llamadas API
import json     # Para parsear respuestas JSON

# Intenta importar librerías de visualización
try:
    import folium
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    import numpy as np
    from branca.element import Template, MacroElement
    folium_installed = True
    print("Bibliotecas de visualización (folium, matplotlib) cargadas.")
except ImportError:
    print("ADVERTENCIA: Folium y/o Matplotlib no están instalados.")
    print("Para visualizar el mapa, por favor instálalos:")
    print("  pip install folium matplotlib numpy requests")
    folium_installed = False

# --- Definir URL del Servicio de RUTA OSRM ---
OSRM_ROUTE_URL = "http://router.project-osrm.org/route/v1/driving/" # URL para obtener geometría

# --- Verificar si se puede proceder con la visualización ---
can_visualize = False
if folium_installed and 'results' in locals() and 'model' in locals():
    term_cond = results.solver.termination_condition
    sol_status_ok = True
    if term_cond == pyo.TerminationCondition.optimal or \
       (term_cond == pyo.TerminationCondition.feasible and sol_status_ok) or \
       (term_cond == pyo.TerminationCondition.maxTimeLimit and sol_status_ok):
        print("Se encontró una solución válida. Procediendo a generar el mapa con rutas por carretera...")
        can_visualize = True
    else:
        print(f"\nEl estado del solver ({term_cond}) no indica una solución utilizable.")
        print("No se generará el mapa.")


# --- Funciones de Visualización ---
if can_visualize:

    # Función extract_routes_and_assignments_for_map
    def extract_routes_and_assignments_for_map(solution_model):
        """Extrae rutas y asignaciones del modelo Pyomo resuelto."""
        routes = {}
        assignments = {}
        for k in solution_model.K:
            assignments[k] = None
            for i in solution_model.I:
                y_val = pyo.value(solution_model.y[i, k], exception=False)
                if y_val is not None and y_val > 0.5:
                    assignments[k] = i; break
        for v_id in solution_model.K:
            start_depot = assignments.get(v_id)
            routes[v_id] = []
            if not start_depot: continue
            departs = any(pyo.value(solution_model.x[start_depot, node, v_id], exception=False) > 0.5
                          for node in solution_model.N if node != start_depot)
            if not departs: continue
            route = [start_depot]; current_node = start_depot
            max_steps = len(solution_model.N) + 2; route_complete = False; error_in_route = False
            for step in range(max_steps):
                next_node = None; found_next = False; possible_next_nodes = []
                for j in solution_model.N:
                    if current_node != j:
                        x_val = pyo.value(solution_model.x[current_node, j, v_id], exception=False)
                        if x_val is not None and x_val > 0.5: possible_next_nodes.append(j); found_next = True
                if len(possible_next_nodes) > 1: next_node = possible_next_nodes[0]
                elif len(possible_next_nodes) == 1: next_node = possible_next_nodes[0]
                if not found_next:
                    if current_node == start_depot and step > 0: route_complete = True
                    else: error_in_route = True
                    break
                route.append(next_node); current_node = next_node
                if current_node == start_depot: route_complete = True; break
                if step == max_steps - 1: error_in_route = True; break
            if route_complete and not error_in_route: routes[v_id] = route
            elif route: routes[v_id] = [] # Marcar como vacía si hay error
        return routes, assignments

    # Función create_map_base 
    def create_map_base(data_dict):
        coords_dict = data_dict.get('UbicacionNodo'); default_location = [4.6097, -74.0817]; zoom_start = 11
        if not coords_dict: return folium.Map(location=default_location, zoom_start=zoom_start, tiles='CartoDB positron')
        lats = [loc[0] for loc in coords_dict.values() if isinstance(loc, (list, tuple)) and len(loc) == 2]
        longs = [loc[1] for loc in coords_dict.values() if isinstance(loc, (list, tuple)) and len(loc) == 2]
        if not lats or not longs: return folium.Map(location=default_location, zoom_start=zoom_start, tiles='CartoDB positron')
        avg_lat = sum(lats) / len(lats); avg_long = sum(longs) / len(longs)
        lat_span = max(lats) - min(lats); lon_span = max(longs) - min(longs)
        if lat_span < 0.1 and lon_span < 0.1: zoom_start = 14
        elif lat_span < 0.5 and lon_span < 0.5: zoom_start = 12
        else: zoom_start = 11
        return folium.Map(location=[avg_lat, avg_long], zoom_start=zoom_start, tiles='CartoDB positron')


    # Función add_map_legend
    def add_map_legend(m, vehicle_colors):
        items = ''.join(f'<div style="margin-bottom: 5px;"><i style="background:{color}; width:15px; height:15px; float:left; margin-right:8px; border:1px solid grey; border-radius: 50%;"></i>{vehicle}</div>'
                        for vehicle, color in vehicle_colors.items() if color)
        if not items: return
        legend_html = f'''<div style="position: fixed; bottom: 30px; left: 10px; width: auto; max-width: 180px; height: auto; z-index:9999; font-size:12px; background-color: rgba(255, 255, 255, 0.85); padding: 10px; border:1px solid grey; border-radius:8px; box-shadow: 3px 3px 5px rgba(0,0,0,0.3);"><h4 style="margin-top:0; margin-bottom:10px; font-weight:bold; text-align:center;">Vehículos</h4>{items}</div>'''
        legend = MacroElement(); legend._template = Template(legend_html); m.get_root().add_child(legend)


    # Función para plotear rutas y marcadores (MODIFICADA PARA USAR OSRM)
    def plot_routes_and_markers(m, routes, assignments, data_dict, solution_model, vehicle_colors):
        """Dibuja las rutas (usando OSRM para geometría) y los marcadores."""
        coords_dict = data_dict.get('UbicacionNodo', {})
        if not coords_dict:
            print("Error Mapeo: No hay coordenadas ('UbicacionNodo') para plotear.")
            return

        # Ya no recalculamos cmap ni vehicle_colors aquí:
        active_routes = {v:r for v,r in routes.items() if r and len(r)>1}
        num_active_vehicles = len(active_routes)

        # --- 1. Dibujar las RUTAS usando OSRM ---
        print(f"Obteniendo geometría de rutas desde OSRM para {num_active_vehicles} vehículos activos...")
        for i, (v_id, route_nodes) in enumerate(active_routes.items()):
            color = vehicle_colors.get(v_id, '#808080')
            print(f"  Procesando ruta para Vehículo {v_id} ({i+1}/{num_active_vehicles})...")

            # Obtener coordenadas y formatear para OSRM (lon,lat;lon,lat...)
            osrm_coords_str = ""
            route_coords_list = [] # Lista de (lat, lon) para fallback
            valid_coords = True
            for node_id in route_nodes:
                coords = coords_dict.get(node_id)
                if coords and isinstance(coords, (list, tuple)) and len(coords) == 2:
                    lat, lon = coords
                    route_coords_list.append((lat, lon))
                    osrm_coords_str += f"{lon},{lat};" # Formato OSRM: lon,lat
                else:
                    print(f"    ERROR: Coordenada inválida/faltante para nodo {node_id} en ruta {v_id}. No se puede consultar OSRM.")
                    valid_coords = False
                    break
            if not valid_coords or len(route_coords_list) < 2:
                print(f"    Saltando consulta OSRM para ruta de {v_id}.")
                continue

            osrm_coords_str = osrm_coords_str.rstrip(';')

            # Construir URL y parámetros para OSRM
            osrm_request_url = f"{OSRM_ROUTE_URL}{osrm_coords_str}"
            params = {
                'overview': 'full',
                'geometries': 'geojson',
                'steps': 'false',
                'alternatives': 'false',
                # 'annotations': 'false'
            }

            # --- Llamada a OSRM con manejo de errores y delay ---
            route_geojson = None
            try:
                # Introducir un pequeño delay para ser amable con el servidor público
                time.sleep(0.6) # ~1-2 llamadas por segundo máx

                response = requests.get(osrm_request_url, params=params, timeout=15) # Timeout de 15s
                response.raise_for_status() # Lanza error para respuestas 4xx/5xx
                route_data = response.json()

                if route_data.get('code') == 'Ok' and route_data.get('routes'):
                    # Extraer la geometría GeoJSON
                    route_geojson = route_data['routes'][0]['geometry']
                    print(f"      Geometría OSRM obtenida.")
                else:
                    print(f"    ADVERTENCIA: Respuesta OSRM no fue 'Ok' o no contenía rutas para {v_id}. Código: {route_data.get('code')}")

            except requests.exceptions.RequestException as e:
                print(f"    ERROR: Falló la consulta a OSRM para la ruta de {v_id}: {e}")
            except json.JSONDecodeError:
                print(f"    ERROR: No se pudo decodificar la respuesta JSON de OSRM para {v_id}.")
            except Exception as e: # Capturar otros errores inesperados
                print(f"    ERROR inesperado durante consulta OSRM para {v_id}: {e}")

            # --- Dibujar en el mapa ---
            if route_geojson:
                # Dibujar usando GeoJson si OSRM funcionó
                style_function = lambda x, color=color: {
                    'color': color,
                    'weight': 3.5,
                    'opacity': 0.85
                }
                folium.GeoJson(
                    route_geojson,
                    name=f'Ruta {v_id}',
                    style_function=style_function,
                    tooltip=f"Ruta Vehículo {v_id} (OSRM)"
                ).add_to(m)
            else:
                # Fallback: Dibujar línea recta si OSRM falló
                print(f"    FALLBACK: Dibujando línea recta para la ruta de {v_id}.")
                folium.PolyLine(
                    locations=route_coords_list, # Usar la lista (lat, lon)
                    color=color,
                    weight=2.5, # Más delgada para indicar fallback
                    opacity=0.6,
                    dash_array='5, 5', # Línea punteada para indicar fallback
                    tooltip=f"Ruta Vehículo {v_id} (Recta - Falló OSRM)"
                ).add_to(m)

        # --- 2. Añadir MARCADORES para todos los nodos ---
        print("\nAñadiendo marcadores de nodos...")
        all_node_ids = set(solution_model.N)
        for node_id in all_node_ids:
            coords = coords_dict.get(node_id)
            if not coords or not isinstance(coords, (list, tuple)) or len(coords) != 2: continue
            lat, lon = coords
            node_type = 'Cliente' if node_id in solution_model.J else 'Depósito' if node_id in solution_model.I else 'Desconocido'
            demand = 0; marker_color = 'gray'; icon_name = 'question'; vehicle_info = "No Asignado/Visitado"
            popup_text = f"<b>Nodo:</b> {node_id}<br><b>Tipo:</b> {node_type}"

            if node_type == 'Depósito':
                icon_name = 'industry'; vehicles_at_depot = [k for k, dep in assignments.items() if dep == node_id]
                if vehicles_at_depot:
                    marker_color = 'darkblue'; active_vehicles_str = ", ".join([f"{k}{'*' if k in active_routes else ''}" for k in vehicles_at_depot])
                    vehicle_info = f"Base de: {active_vehicles_str} (*=activa)"
                else: marker_color = 'lightblue'; vehicle_info = "No utilizado"
                popup_text += f"<br>{vehicle_info}"
            elif node_type == 'Cliente':
                icon_name = 'user'; vehicle_visiting = None
                try: demand = solution_model.D[node_id].value
                except: pass # Ignorar error de demanda si no existe
                for v_id_active in active_routes.keys():
                    is_visited = any((pyo.value(solution_model.x[u, node_id, v_id_active], exception=False) or 0) > 0.5 for u in solution_model.N if u != node_id)
                    if is_visited: vehicle_visiting = v_id_active; break
                if vehicle_visiting: marker_color = vehicle_colors.get(vehicle_visiting, 'green'); vehicle_info = f"Servido por {vehicle_visiting}"
                else: marker_color = 'red'; vehicle_info = "NO SERVIDO" # Simplificado
                popup_text += f"<br><b>Demanda:</b> {demand:.2f}"
                popup_text += f"<br>{vehicle_info}"
            else: marker_color = 'black'; icon_name = 'times'

            icon = folium.Icon(color=marker_color, icon=icon_name, prefix='fa')
            folium.Marker(location=[lat, lon], icon=icon, popup=folium.Popup(popup_text, max_width=250, sticky=True), tooltip=f"Nodo {node_id} ({node_type})").add_to(m)

        # --- 3. Añadir la leyenda ---
        add_map_legend(m, vehicle_colors)

    
def generate_route_map_multiple(models_list, case_number):
    """
    Genera un único mapa con todas las rutas superpuestas y guarda
    el HTML en la estructura de carpetas y nombres que ya tienes.
    """
    if not folium_installed:
        print("ERROR: Folium/Matplotlib no disponibles. Instala las dependencias.")
        return
    
    # --- Generación y Visualización del Mapa ---
    print("Generando mapa con rutas por carretera...")

    # 1) Filtrar submodelos con solución
    usable = []
    for m in models_list:
        term = results.solver.termination_condition
        if term in (
            pyo.TerminationCondition.optimal,
            pyo.TerminationCondition.feasible,
            pyo.TerminationCondition.maxTimeLimit
        ):
            usable.append(m)
    if not usable:
        print("Ningún submodelo utilizable.")
        return

    # 2) Extraer todas las rutas para cada modelo
    all_routes = {}
    all_assigns = {}
    for m in usable:
        routes, assigns = extract_routes_and_assignments_for_map(m)
        # fusionar en diccionario global
        all_routes.update(routes)
        all_assigns.update(assigns)

    # 3) Construir vehicle_colors global
    active_vehicle_ids = sorted(k for k, r in all_routes.items() if r and len(r) > 1)
    num = len(active_vehicle_ids)
    cmap = plt.cm.get_cmap('tab10', num if num>0 else 1)
    vehicle_colors = {
        v_id: mcolors.rgb2hex(cmap(idx % cmap.N))
        for idx, v_id in enumerate(active_vehicle_ids)
    }

    # Preparar mapa base
    data_dict = globals()[f'data{case_number}']
    print(f"Ubicación de nodos: {data_dict}")
    folium_map = create_map_base(data_dict)

    # Agregar rutas y marcadores de cada submodelo
    for idx, model in enumerate(models_list, start=1):
        print(f"\n--- Procesando submodelo {idx}/{len(models_list)} ---")
        routes, assignments = extract_routes_and_assignments_for_map(model)
        plot_routes_and_markers(folium_map, routes, assignments, data_dict, model, vehicle_colors)

    # Guardar output
    group_name = os.getenv('GROUP_NAME', 'Grupo10')

    output_dir = f"{group_name}-caso-{case_number}-results"
    os.makedirs(output_dir, exist_ok=True)
    map_filename = os.path.join(
        output_dir,
        f"{group_name}-caso-{case_number}-mapa_rutas_carretera.html"
    )
    folium_map.save(map_filename)
    print(f"\nMapa combinado guardado en: {os.path.abspath(map_filename)}")


Bibliotecas de visualización (folium, matplotlib) cargadas.
Se encontró una solución válida. Procediendo a generar el mapa con rutas por carretera...


In [134]:
generate_route_map_multiple([modelos_pyomo['case1']], 1)
generate_route_map_multiple([modelos_pyomo['case2']], 2)
generate_route_map_multiple(modelos_pyomo_case3, 3)

Generando mapa con rutas por carretera...
Ubicación de nodos: {'N': ['D1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24'], 'UbicacionNodo': {'D1': (4.75021190869025, -74.08124218159384), 'C1': (4.59795431125545, -74.09893796560621), 'C2': (4.687820646838871, -74.07557103763986), 'C3': (4.70949446000624, -74.10708524062704), 'C4': (4.605029068682624, -74.09727965657427), 'C5': (4.648463876533332, -74.16464148202755), 'C6': (4.662137416953968, -74.12083799988112), 'C7': (4.697499030379109, -74.02213076607309), 'C8': (4.649416884236942, -74.17207549744595), 'C9': (4.606310650273935, -74.15615257246444), 'C10': (4.557379705282216, -74.09041145358674), 'C11': (4.591594072172954, -74.17802255204528), 'C12': (4.7564172406324055, -74.1015410917749), 'C13': (4.646217006050524, -74.09690889182339), 'C14': (4.725912125314368, -74.1219200708342), 'C15': (4.604168478560718, -74.0942948461

C:\Users\nicol\AppData\Local\Temp\ipykernel_12168\984971049.py:280: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('tab10', num if num>0 else 1)


      Geometría OSRM obtenida.
  Procesando ruta para Vehículo V10 (2/3)...
      Geometría OSRM obtenida.
  Procesando ruta para Vehículo V21 (3/3)...
      Geometría OSRM obtenida.

Añadiendo marcadores de nodos...


C:\Users\nicol\AppData\Local\Temp\ipykernel_12168\984971049.py:235: UserWarning: color argument of Icon should be one of: {'lightblue', 'pink', 'darkgreen', 'lightgreen', 'gray', 'orange', 'red', 'blue', 'cadetblue', 'lightred', 'lightgray', 'white', 'purple', 'darkred', 'black', 'darkblue', 'darkpurple', 'green', 'beige'}.
  icon = folium.Icon(color=marker_color, icon=icon_name, prefix='fa')



Mapa combinado guardado en: c:\Users\nicol\Desktop\CodingProjects\ISIS3302-ProyectoMOS\Grupo10-caso-1-results\Grupo10-caso-1-mapa_rutas_carretera.html
Generando mapa con rutas por carretera...
Ubicación de nodos: {'N': ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'], 'UbicacionNodo': {'D1': (4.75021190869025, -74.08124218159384), 'D2': (4.5363832206427785, -74.10993358606953), 'D3': (4.792925960208614, -74.03854814565923), 'D4': (4.72167778077445, -74.06706883098641), 'D5': (4.607707046760958, -74.13826337931849), 'D6': (4.650463053612691, -74.12400186370824), 'D7': (4.621911772492814, -74.09561875464892), 'D8': (4.678960680833056, -74.10975623736951), 'D9': (4.735973062153282, -74.09547235719887), 'D10': (4.550640992537941, -74.10991610076434), 'D11': (4.664702960902753, -74.10977422186991), 'D12': (4.5791740634103135, -74.12408925943333), 'C1': (4.632552840424734, -74.19699184741948), 'C2': (4.60132821